In [1]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, Float, DateTime, String, ForeignKey, MetaData, Engine

In [2]:
def load_data_from_silver(filename:str, reset_index:bool=True) -> pd.DataFrame:
    """ This function is for loading parquet files from the Datasets/Silver folder. """
    dataframe = pd.read_parquet(f'Datasets/Silver/parquet/{filename}.parquet')
    return dataframe.reset_index() if reset_index else dataframe

def load_data_to_sqlalchemy(dataframe: pd.DataFrame, table_name: str, engine:Engine) -> None:
    """ This function is for importing data from parquet to sql database. """
    dataframe.to_sql(table_name, con=engine, if_exists='append', index=False)
    
def load_data_to_gold(dataframe:pd.DataFrame, filename:str) -> None:
    dataframe.to_parquet(f'Datasets/Gold/{filename}.parquet', engine='pyarrow')
    return

In [3]:
# Load all dataset to dataframe
recipe = load_data_from_silver('recipe')
recipe_category = load_data_from_silver('recipe_category')
recipe_tag = load_data_from_silver('recipe_tag')
recipe_course = load_data_from_silver('recipe_course')
recipe_cuisine = load_data_from_silver('recipe_cuisine')
recipe_ingredient = load_data_from_silver('recipe_ingredient')
recipe_instruction = load_data_from_silver('recipe_instruction')
recipe_nutrition = load_data_from_silver('recipe_nutrition')
category = load_data_from_silver('category')
tag = load_data_from_silver('tag')
course = load_data_from_silver('course')
cuisine = load_data_from_silver('cuisine')
ingredient = load_data_from_silver('ingredient')
nutrition = load_data_from_silver('nutrition')
unit = load_data_from_silver('unit')

In [4]:
# Create sqlalchemy engine instance / Set echo to True to enable logging.
engine = create_engine(f'sqlite:///Datasets/Silver/pinoyfoodblog.db', echo=False)
metadata = MetaData()

In [5]:
# Prepare all tables without dependencies
category_tbl = Table('category', metadata,
    Column('category_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

tag_tbl = Table('tag', metadata,
    Column('tag_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

course_tbl = Table('course', metadata,
    Column('course_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

cuisine_tbl = Table('cuisine', metadata,
    Column('cuisine_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

ingredient_tbl = Table('ingredient', metadata,
    Column('ingredient_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

nutrition_tbl = Table('nutrition', metadata,
    Column('nutrition_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

unit_tbl = Table('unit', metadata,
    Column('unit_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False)
)

recipe_tbl = Table('recipe', metadata,
    Column('recipe_id', Integer, primary_key=True),
    Column('name', String(255), nullable=False),
    Column('link', String(255), nullable=False),
    Column('thumbnail', String(255), nullable=True),
    Column('description', String(255), nullable=True),
    Column('publish', DateTime, nullable=True),
    Column('modified', DateTime, nullable=True),
    Column('prep_time', Integer, nullable=True),
    Column('cook_time', Integer, nullable=True),
    Column('custom_time', Float, nullable=True),
    Column('good_for', Integer, nullable=True),
)

In [6]:
# Prepare all tables with dependencies
recipe_category_tbl = Table('recipe_category', metadata,
    Column('recipe_category_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('category_id', Integer, ForeignKey('category.category_id'), nullable=False)
)

recipe_tag_tbl = Table('recipe_tag', metadata,
    Column('recipe_tag_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('tag_id', Integer, ForeignKey('tag.tag_id'), nullable=False)
)

recipe_course_tbl = Table('recipe_course', metadata,
    Column('recipe_course_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('course_id', Integer, ForeignKey('course.course_id'), nullable=False)
)

recipe_cuisine_tbl = Table('recipe_cuisine', metadata,
    Column('recipe_cuisine_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('cuisine_id', Integer, ForeignKey('cuisine.cuisine_id'), nullable=False)
)

recipe_ingredient_tbl = Table('recipe_ingredient', metadata,
    Column('recipe_ingredient_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('ingredient_id', Integer, ForeignKey('ingredient.ingredient_id'), nullable=False),
    Column('min_amount', Float, nullable=False),
    Column('max_amount', Float, nullable=True),
    Column('unit_id', Integer, ForeignKey('unit.unit_id'), nullable=False),
    Column('notes', String(255), nullable=True),
)

recipe_instruction_tbl = Table('recipe_instruction', metadata,
    Column('recipe_instruction_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('step', Integer, nullable=False),
    Column('instruction', String(255), nullable=True)
)

recipe_nutrition_tbl = Table('recipe_nutrition', metadata,
    Column('recipe_nutrition_id', Integer, primary_key=True),
    Column('recipe_id', Integer, ForeignKey('recipe.recipe_id'), nullable=False),
    Column('nutrition_id', Integer, ForeignKey('nutrition.nutrition_id'), nullable=False),
    Column('value', Float, nullable=True),
    Column('unit_id', Integer, ForeignKey('unit.unit_id'), nullable=False),
    Column('daily', Float, nullable=True),
)

In [7]:
# Create all table structure stored from the metadata
metadata.create_all(engine)

In [8]:
# Load data from dataframe to database - Silver
load_data_to_sqlalchemy(category, 'category', engine)
load_data_to_sqlalchemy(tag, 'tag', engine)
load_data_to_sqlalchemy(course, 'course', engine)
load_data_to_sqlalchemy(cuisine, 'cuisine', engine)
load_data_to_sqlalchemy(ingredient, 'ingredient', engine)
load_data_to_sqlalchemy(nutrition, 'nutrition', engine)
load_data_to_sqlalchemy(unit, 'unit', engine)
load_data_to_sqlalchemy(recipe, 'recipe', engine)

load_data_to_sqlalchemy(recipe_category, 'recipe_category', engine)
load_data_to_sqlalchemy(recipe_tag, 'recipe_tag', engine)
load_data_to_sqlalchemy(recipe_course, 'recipe_course', engine)
load_data_to_sqlalchemy(recipe_cuisine, 'recipe_cuisine', engine)
load_data_to_sqlalchemy(recipe_ingredient, 'recipe_ingredient', engine)
load_data_to_sqlalchemy(recipe_instruction, 'recipe_instruction', engine)
load_data_to_sqlalchemy(recipe_nutrition, 'recipe_nutrition', engine)

In [9]:
quickiest_food_to_prepare_and_cook = pd.read_sql_query("""
WITH recipe_valid_time AS (
    SELECT 
    	name,
    	prep_time+cook_time+custom_time AS total_time,
    	(prep_time+cook_time+custom_time)/good_for AS minute_per_pax,
		link
    FROM recipe AS r 
    WHERE 1=1
    	AND (prep_time != 0 OR cook_time != 0 OR custom_time != 0)
    	AND good_for != 0
)
SELECT * FROM recipe_valid_time ORDER BY minute_per_pax ASC
""", engine)

load_data_to_gold(quickiest_food_to_prepare_and_cook, 'quickiest_food_to_prepare_and_cook')

In [10]:
quickiest_food_to_prepare_and_cook.head()

,name,total_time,minute_per_pax,link
0,filipino vinegar dipping sauce,15.0,0.750000,https://panlasangpinoy.com/filipino-vinegar-di...
1,cheese pimiento sandwich recipe,10.0,1.250000,https://panlasangpinoy.com/cheese-pimiento-san...
2,bicol express gising-gising recipe,5.0,1.250000,https://panlasangpinoy.com/bicol-express-gisin...
3,how to make macaroni salad for the holidays,10.0,1.666667,https://panlasangpinoy.com/how-to-make-macaron...
4,java rice recipe,12.0,2.000000,https://panlasangpinoy.com/java-rice-recipe/


In [11]:
def most_nutrition_and_quickiest_food_to_prepare_and_cook(nutrition_id:int) -> pd.DataFrame:
    result = pd.read_sql_query(f"""                                                                     
    WITH recipe_temp AS (
        SELECT 
            recipe_id,
            name, 
            link, 
            prep_time+cook_time+custom_time AS total_time, 
            good_for 
        FROM recipe AS r
        WHERE total_time != 0 AND good_for != 0
    )
        SELECT 
            rn.recipe_id,
            r.name,
            rn.value,
            u.name AS value_unit,
            r.total_time AS cook_and_prep_time_in_mins, 
            r.good_for,
            rn.value/r.good_for AS nutrition_per_pax,
            (rn.value/r.good_for)/r.total_time AS nutrition_per_minute,
            r.link
        FROM recipe_nutrition AS rn
        INNER JOIN 
            recipe_temp AS r ON rn.recipe_id = r.recipe_id
        INNER JOIN 
            nutrition AS n ON rn.nutrition_id  = n.nutrition_id
        INNER JOIN 
            unit AS u ON rn.unit_id = u.unit_id
        WHERE rn.nutrition_id = {nutrition_id}
        ORDER BY nutrition_per_minute DESC
    """, engine)
    return result

calories = 1000 # Calories unit_id
load_data_to_gold(most_nutrition_and_quickiest_food_to_prepare_and_cook(calories), 'most_nutrition_and_quickiest_food_to_prepare_and_cook')

In [12]:
most_nutrition_and_quickiest_food_to_prepare_and_cook(calories).head()

,recipe_id,name,value,value_unit,cook_and_prep_time_in_mins,good_for,nutrition_per_pax,nutrition_per_minute,link
0,1302,crispy eggplant fries,1963.0,kilocalorie,10.0,2,981.500000,98.150000,https://panlasangpinoy.com/crispy-eggplant-fries/
1,1266,bicol express gising-gising recipe,1142.0,kilocalorie,5.0,4,285.500000,57.100000,https://panlasangpinoy.com/bicol-express-gisin...
2,2591,chicken pasta alfredo recipe,7129.0,kilocalorie,50.0,4,1782.250000,35.645000,https://panlasangpinoy.com/chicken-pasta-alfre...
3,1029,cheesy lumpia,6987.0,kilocalorie,40.0,5,1397.400000,34.935000,https://panlasangpinoy.com/cheesy-lumpia/
4,1218,how to make macaroni salad for the holidays,2041.0,kilocalorie,10.0,6,340.166667,34.016667,https://panlasangpinoy.com/how-to-make-macaron...
